# Percepción Computacional - Actividad 2

## Preparación del entorno 

In [ ]:
from pathlib import Path
from typing import List

import session_info
import numpy as np
import matplotlib.pyplot as plt 
from skimage import exposure
from skimage.io import imread
from skimage.color import rgb2gray
from scipy.ndimage import laplace
from scipy.signal import medfilt2d
from cv2 import Sobel, Canny, threshold, erode, dilate, morphologyEx
from cv2 import CV_64F, THRESH_BINARY, THRESH_OTSU, MORPH_OPEN, MORPH_CLOSE, MORPH_GRADIENT

In [ ]:
DATA_DIR = Path("../imagenes/")

SCIENCE_DIR = DATA_DIR / "cientifico"
MEDICAL_DIR = DATA_DIR / "medico"
INDUSTRIAL_DIR = DATA_DIR / "industrial"
SATELITAL_DIR = DATA_DIR / "satelital"

In [ ]:
session_info.show()

## Funciones útiles

In [ ]:
def read_images(directory_path: Path) -> List[Path]:
    image_paths = []
    
    for image_path in directory_path.glob("*"):
        image_paths.append(image_path)
    
    image_paths.sort()
    
    for image_path in image_paths:
        print(image_path)

    return image_paths
        

def plot(img: np.ndarray, title: str=""):
    plt.figure(figsize=(14,8))
    plt.axis('off')
    
    if isinstance(img, np.ndarray) and len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(img)

    plt.title(title)
    
    
    plt.show()


def process_image(image_path: Path, increase_contrast: bool=False):
    original_image = imread(image_path)
    plot(original_image, "Imagen original")

    if increase_contrast:
        equalized_image = exposure.equalize_hist(original_image)
        gamma_corrected = exposure.adjust_gamma(equalized_image, 2)
        gamma_and_log_corrected = exposure.adjust_log(gamma_corrected, 1)

        original_image = gamma_and_log_corrected        
        plot(original_image, "Imagen original con equalizada, con correción de gamma y filtrado logaritmico")
        
    if len(original_image.shape) == 3:
        # Si tiene alfa channel, lo descartamos
        if original_image.shape[-1] == 4:
            original_image = original_image[:,:,:-1]

        grayscale_image = rgb2gray(original_image)
        grayscale_image = (grayscale_image * 255).astype(np.uint8)
    
    else:
        grayscale_image = original_image
    
    plot(grayscale_image, "Imagen en escala de grises")

    # Filtrado espacial lineal - suavizado
    filtered_grayscale_image = medfilt2d(grayscale_image, [3,3])
    plot(filtered_grayscale_image, "Imagen filtrada espacialmente mediante la mediana")

    # Realzamos los bordes 
    sobel_horizontal_grayscale_image = Sobel(src=grayscale_image, ddepth=CV_64F, dx=0, dy=1, ksize=3) 
    sobel_vertical_grayscale_image = Sobel(src=grayscale_image, ddepth=CV_64F, dx=1, dy=0, ksize=3)
    sobel_grayscale_image = (sobel_horizontal_grayscale_image + sobel_vertical_grayscale_image) / 2
    plot(sobel_grayscale_image, "Bordes resaltados mediante filtrado de Sobel")

    # Realzamos los bordes diagonales
    sobel_diagonal_grayscale_image = Sobel(src=grayscale_image, ddepth=CV_64F, dx=1, dy=1, ksize=3) 
    plot(sobel_diagonal_grayscale_image, "Bordes diagonales resaltados mediante filtrado de Sobel")

    # Realzamos los bordes mediante 
    canny_grayscale_image = Canny(image=grayscale_image, threshold1=100, threshold2=200)
    plot(canny_grayscale_image, "Bordes resaltados mediante filtrado de Canny")

    # Realce de detalles finos
    laplace_filtered_grayscale_image = laplace(grayscale_image)
    plot(laplace_filtered_grayscale_image, "Detalles relaltados mediante filtrado con filtro de Laplace")

    # Binarización de la imagen
    # Usamos el método de Otsu para encontrar el threshold de la binaziración, es el default de matlab
    binarized_image = threshold(grayscale_image, 0, 255, THRESH_BINARY+THRESH_OTSU)[1]
    plot(binarized_image, "Imagen binarizada (usando el método de Otsu para encontrar el punto de corte)")

    structurant_element = np.ones((3,3), np.uint8)
    eroded_binarized_image = erode(binarized_image, structurant_element, iterations = 1)
    dilated_binarized_image = dilate(binarized_image, structurant_element, iterations = 1)

    plot(eroded_binarized_image, "Imagen binaria erosionada")
    plot(dilated_binarized_image, "Imagen binaria dilatada")
    
    # Dilatación y erosión de una imagen en escala de grises
    eroded_image = erode(grayscale_image, structurant_element, iterations = 1)
    dilated_image = dilate(grayscale_image, structurant_element, iterations = 1)
    opened_image = morphologyEx(grayscale_image, MORPH_OPEN, structurant_element)
    closed_image = morphologyEx(grayscale_image, MORPH_CLOSE, structurant_element)

    plot(eroded_image, "Imagen en escala de grises erosionada")
    plot(dilated_image, "Imagen en escala de grises dilatada")
    plot(opened_image, "Imagen en escala de grises abierta")
    plot(closed_image, "Imagen en escala de grises cerrada")

    # Gradientes y operadores basados en ellos
    internal_gradient = grayscale_image - eroded_image
    external_gradient = dilated_image - grayscale_image
    gradient = dilated_image - eroded_image
    morphological_gradient = morphologyEx(grayscale_image, MORPH_GRADIENT, structurant_element)
    white_top_hat = grayscale_image - opened_image
    black_top_hat = closed_image - grayscale_image
    top_hat = closed_image - opened_image

    plot(internal_gradient, "Gradiente Interno")
    plot(external_gradient, "Gradiente Externo")
    plot(gradient, "Gradiente")
    plot(morphological_gradient, "Gradiente Morfológico")
    plot(white_top_hat, "White Top Hat")
    plot(internal_gradient, "Black Top Hat")
    plot(internal_gradient, "Top Hat")
    plot(internal_gradient, "Gradiente Interno")
    plot(internal_gradient, "Gradiente Interno")

    # Detección de contornos
    open_countour = opened_image - eroded_image
    dilated_countour = dilated_image - eroded_image

    plot(open_countour, "Contornos mediante apertura")
    plot(dilated_countour, "Contornos mediante dilatación")

## Casos Científicos (Microscopio electrónico)

In [ ]:
image_paths = read_images(SCIENCE_DIR)

In [ ]:
# Ref: https://www.sciencephoto.com/media/668923/view/zinc-oxide-crystals-sem
process_image(image_paths[0])

In [ ]:
# Ref: https://en.wikipedia.org/wiki/Scanning_electron_microscope#/media/File:Surface_of_a_kidney_stone.jpg
process_image(image_paths[1])

## Casos Médicos (CT-Scan)

In [ ]:
image_paths = read_images(MEDICAL_DIR)

In [ ]:
# Ref: https://www.sciencedirect.com/science/article/pii/S0378603X15000054?ref=pdf_download&fr=RR-2&rr=84602195fe2056fd

process_image(image_paths[0], True)

## Casos Industrial (Imagen termográfica)

In [ ]:
image_paths = read_images(INDUSTRIAL_DIR)

In [ ]:
process_image(image_paths[0])

## Caso Satelital 

In [ ]:
image_paths = read_images(SATELITAL_DIR)

In [ ]:
process_image(image_paths[0])